In [ ]:
!pip install --upgrade gensim -qq 
!pip install transformers -qq 
!pip install tensorflow_addons -qq 
!pip install glove -qq 
!pip install tensorflow -qq 
!pip install nltk -qq 
!pip install keras -qq 
!pip install vaderSentiment -qq 
!pip install emot
!pip install glove_python
!pip install glove
!pip install glove-python-binary


In [18]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
import numpy as np
import scipy
import tensorflow_hub as hub
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.sequence import pad_sequences
from gensim.models import KeyedVectors
from transformers import BertTokenizer
import transformers
import html #Messing with HTML content, like &amp;
import string #String Processing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import  accuracy_score
from tensorflow.keras.layers import Dense, Bidirectional, LSTM, GRU, Embedding
import torch
from transformers import AutoTokenizer, AutoModel
import tensorflow
from tensorflow.keras import layers
from tensorflow.keras.regularizers import L1L2
import pickle
from emot.emo_unicode import UNICODE_EMOJI
from matplotlib import pyplot as plt
import seaborn as sns
from random import seed
from random import randrange
from csv import reader
from math import sqrt
from glove import Corpus, Glove


# precision-recall-f1

In [3]:

from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))



# Preproccessing + Getting X

In [ ]:
!gdown --id 1DXDsCPZDUY3G828cEQnG40Uci3tJ1PUi
!gdown --id 155bZkqIDpy0y5RRP3yx4PFEMIlGDVKD6
!gdown --id 1dzpXfikb2QyS0x61PwQ3EiPQ7Fi_k3Rb
!gdown --id 1oEKK3cz9ESY6HR-_gudDFrWt_e3ZkQG0
!gdown --id 1rlQDiv-H14DXBh2Uvx61uw9XknjJFee8
!gdown --id 1i27GazTEa9VTJh7VpJ0CY9Ce-mVt-IuY



In [6]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F" # emoticons
                           u"\U0001F300-\U0001F5FF" # symbols & pictographs
                           u"\U0001F680-\U0001F6FF" # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF" # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

remove_emoji("Hilarious 😂! The feeling of making a sale 😎, The feeling of actually fulfilling orders 😒")

'Hilarious ! The feeling of making a sale , The feeling of actually fulfilling orders '

In [14]:
nltk.download('stopwords')
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 

punctuations = string.punctuation #List of punctuations to remove
print(punctuations) #See the punctuations the string library has

STOP = stopwords.words("english") #Get the NLTK stopwords
print(STOP) #See what NLTK considers stopwords

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
def convert_emojis(text):
    for emot in UNICODE_EMOJI:
        text = text.replace(emot, "_".join(UNICODE_EMOJI[emot].replace(",","").replace(":","").split()))
    return text
    
# Example
text1 = "Hilarious 😂. The feeling of making a sale 😎, The feeling of actually fulfilling orders 😒"
convert_emojis(text1)

'Hilarious face_with_tears_of_joy. The feeling of making a sale smiling_face_with_sunglasses, The feeling of actually fulfilling orders unamused_face'

In [15]:
#CleanTweets: parses the tweets and removes punctuation, stop words, digits, and links.
#Input: the list of tweets that need parsing
#Output: the parsed tweets
def cleanTweets(tweetParse):
    for i in range(0,len(tweetParse)):
        tweet = tweetParse[i] #Putting the tweet into a variable so that it is not calling tweetParse[i] over and over
        tweet = html.unescape(tweet) #Removes leftover HTML elements, such as &amp;
        tweet = re.sub(r"@\w+", " ", tweet) #Completely removes @'s, as other peoples' usernames mean nothing
        tweet = re.sub(r"http\S+", " ", tweet) #Removes links, as links provide no data in tweet analysis in themselves
        tweet = "".join([punc for punc in tweet if not punc in punctuations]) #Removes the punctuation defined above
        tweet = tweet.lower() #Turning the tweets lowercase real quick for later use
        tweet = stemmer.stem(tweet)
        tweet = convert_emojis(tweet)
        tweetWord = tweet.split() #Splits the tweet into individual words
        tweetParse[i] = "".join([word + " " for word in tweetWord if not word in STOP]) #Checks if the words are stop words
    return tweetParse #Returns the parsed tweets


In [16]:
# this function reads the corpus and returns its tweets, Ids and labels
def read_document(path):
  labels = []  
  tmp = ' '
  tweets = []
  ids = []
  df = pd.read_excel(path)
  ids = df.id
  labels = df.label
  df["cleanTweet"] = cleanTweets(df["tweet"].copy()) #Clean the training tweets
  df.head() #Take a look at the dataset
  return  df.cleanTweet, labels, df


In [ ]:
train_tweets, train_labels, train_df = read_document("/content/Constraint_English_Train.xlsx")
test_tweets, test_labels, test_df = read_document("/content/english_test_with_labels.xlsx")
val_tweets, val_labels, val_df = read_document("/content/Constraint_English_Val.xlsx")
add_tweets = [ train_tweets, val_tweets, test_tweets]
all_tweets = pd.concat(add_tweets)


In [ ]:
# Find the best lenght for padding
tweets = [tweet for tweet in train_df.cleanTweet]
max_len = 0
tweets_len = []
for tweet in tweets:
    tweets_len.append(len(tweet.split()))
    max_len = max(len(tweet.split()), max_len)

print('Number of titles:', len(tweets))
print('Max length of the titles:', max_len)
print('Mean length of the titles:', np.mean(tweets_len))
plt.figure(figsize=(20,5))
g = sns.countplot(x=tweets_len)
g.set_xticklabels(g.get_xticklabels(), rotation=50)
plt.show()

# Getting Y

In [ ]:
def label_encode(label):
  
    if label == 'real':
        return 1
    if label == "fake":
        return 0


In [ ]:
encoded_train = [label_encode(label) for label in train_labels]
encoded_val = [label_encode(label) for label in val_labels]
encoded_test = [label_encode(label) for label in test_labels]



In [ ]:
y_val = np.array(encoded_val)
y_train = np.array(encoded_train)
y_test = np.array(encoded_test)

**# Word Embedding**

# word2vec and glove embedding

In [ ]:
# create input for gensim word2vec model
def get_sentences(tweets):
  tweetBYword = []
  for tweet in tweets:
    tokens = tweet.split()
    tokens = [word for word in tokens]
    tweetBYword.append(tokens)
  return tweetBYword


In [ ]:
tweetBYword= get_sentences(all_tweets)


In [ ]:
# sg: Training algorithm: 1 for skip-gram; otherwise CBOW.
# min_count: Ignores all words with total frequency lower than this.
# window: Maximum distance between the current and predicted word within a sentence.
# size: Dimensionality of the word vectors.

model_w2v = Word2Vec(tweetBYword,min_count = 0, vector_size=100, window=10, sg=0, batch_words = 2)

In [ ]:
# save the model
# model_w2v.save("/content/word2vec.model")

In [ ]:
# load the model
# model_w2v = Word2Vec.load("/content/word2vec.model")

In [ ]:
#importing the glove library
from glove import Corpus, Glove
# creating a corpus object
corpus = Corpus() 
#training the corpus to generate the co occurence matrix which is used in GloVe
corpus.fit(tweetBYword, window=10)
#creating a Glove object which will use the matrix created in the above lines to create embeddings
#We can set the learning rate as it uses Gradient Descent and number of components
glove = Glove(no_components=150, learning_rate=0.05)
 
glove.fit(corpus.matrix, epochs=30, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)
glove.save('glove.model')

In [ ]:
gl = glove.load("glove.model")


In [ ]:
def get_max_length(tweet):
    """
    get max token counts from train data, 
    so we use this number as fixed length input to RNN cell
    """
    max_length = 0
    for row in tweet:
        if len(row.split(" ")) > max_length:
            max_length = len(row.split(" "))
    return max_length

def get_word2vec_enc(tweets):
    """
    get word2vec value for each word in sentence.
    concatenate word in numpy array, so we can use it as RNN input
    """
    model_w2v = Word2Vec.load("/content/word2vec.model")
    encoded_tweets = []
    for tweet in tweets:
        tokens = tweet.split(" ")
        word2vec_embeddings = []
        for token in tokens:
            if token in model_w2v.wv.key_to_index:
                 word2vec_embeddings.append(model_w2v.wv[token])
        encoded_tweets.append(word2vec_embeddings)
    return encoded_tweets
        

def get_glove_enc(tweets):
    """
    get word2vec value for each word in sentence.
    concatenate word in numpy array, so we can use it as RNN input
    """
    encoded_tweets = []
    for tweet in tweets:
        tokens = tweet.split(" ")
        glove_embeddings = []
        for token in tokens:
            if token in glove.dictionary.keys():
                 glove_embedding = glove.word_vectors[glove.dictionary[token]]
                 glove_embeddings.append(glove_embedding)
        encoded_tweets.append(glove_embeddings)
    return encoded_tweets
        


def label_encode(label):
    """
    return one hot encoding for Y value
    """
    if label == 'real':
        return 0
    else:
        return 1
    
def GetX(tweets, labels):
  
    # encode text value to numeric value
    encoded_tweets_W2V = get_word2vec_enc(tweets)
    encoded_tweets_Glove = get_glove_enc(tweets)
    # padded len to maxlen
    padded_encoded_tweets_W2V =pad_sequences(encoded_tweets_W2V, padding = 'pre', maxlen = 81, dtype='float32')
    padded_encoded_tweets_Glove =pad_sequences(encoded_tweets_Glove, padding = 'pre', maxlen = 81, dtype='float32')
    encoded_label = [label_encode(label) for label in labels]
    Y = np.array(encoded_label)
    X_w2v = np.array(padded_encoded_tweets_W2V)
    X_glove = np.array(padded_encoded_tweets_Glove)
    return X_w2v,X_glove

In [ ]:
X_train_w2v,X_train_glove = GetX(train_tweets, train_labels)
X_test_w2v,X_test_glove = GetX(test_tweets, test_labels)
X_val_w2v,X_val_glove = GetX(val_tweets, val_labels)

In [ ]:
X_train_avg_w2v = tf.keras.layers.Lambda(lambda x: tf.keras.backend.mean(x, axis=1))(X_train_w2v)
X_test_avg_w2v = tf.keras.layers.Lambda(lambda x: tf.keras.backend.mean(x, axis=1))(X_test_w2v)
X_val_avg_w2v = tf.keras.layers.Lambda(lambda x: tf.keras.backend.mean(x, axis=1))(X_val_w2v)


In [ ]:
X_train_avg_glove= tf.keras.layers.Lambda(lambda x: tf.keras.backend.mean(x, axis=1))(X_train_glove)
X_test_avg_glove = tf.keras.layers.Lambda(lambda x: tf.keras.backend.mean(x, axis=1))(X_test_glove)
X_val_avg_glove = tf.keras.layers.Lambda(lambda x: tf.keras.backend.mean(x, axis=1))(X_val_glove)

#tf-idf word embedding

In [ ]:
feature_extraction = TfidfVectorizer()

In [ ]:
train_test = [train_tweets, test_tweets]
train_test_tweets= pd.concat(train_test)
X = feature_extraction.fit_transform(train_test_tweets)


In [ ]:
X_train_tfidf  = X[:6420]
X_test_tfidf = X[6420:]

# transformer word embedding

In [ ]:

# !pip install transformers 


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
path = "digitalepidemiologylab/covid-twitter-bert-v2"
# path = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(path)
model_embedding = AutoModel.from_pretrained(path).to(device)


Downloading:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/421 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.35G [00:00<?, ?B/s]

Some weights of the model checkpoint at digitalepidemiologylab/covid-twitter-bert-v2 were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
model_embedding.eval()

In [ ]:

def get_embd_tr(tweets):
  # for j in range(4):
  # print(i, i+4)
  for i in range(0, len(tweets), 2):
    if (i == 0):
      tokenized_tweets = tokenizer(tweets.tolist()[:2], pad_to_max_length=True, max_length=81, truncation = True, return_tensors="pt")
      tokenized_tweets= {k:torch.tensor(v).to(device) for k,v in tokenized_tweets.items()}
      with torch.no_grad():
        hidden_tweets = model_embedding(**tokenized_tweets, output_hidden_states=True) #dim : [batch_size(nr_sentences), tokens, emb_dim]
      hidden_states = hidden_tweets.hidden_states[22]
      cls_train = hidden_tweets.last_hidden_state[:,0,:]

    
    else:
      cls_train = hidden_train.hidden_states
    
      tokenized_tweets = tokenizer(tweets.tolist()[i:i+2], pad_to_max_length=True, max_length=81, truncation = True, return_tensors="pt")

      tokenized_tweets = {k:torch.tensor(v).to(device) for k,v in tokenized_tweets.items()}
      with torch.no_grad():
        hidden_tweets = model_embedding(**tokenized_tweets,output_hidden_states=True) #dim : [batch_size(nr_sentences), tokens, emb_dim]
      
      hidden_state = hidden_tweets.hidden_states[22]
      cl_train = hidden_tweets.last_hidden_state[:,0,:]

      hidden_states = torch.cat((hidden_states, hidden_state),0)
      cls_train = torch.cat((cls_train, cl_train),0)

  return cls_train, hidden_states

In [ ]:
cls_train, hidden_states_train = get_embd_tr(train_tweets)
x_cls_train = cls_train.to("cpu")
x_hiddenstates_train = hidden_states_train.to("cpu").numpy()

torch.cuda.empty_cache()

cls_val, hidden_states_val = get_embd_tr(val_tweets)
x_cls_val = cls_val.to("cpu")
x_hiddenstates_val = hidden_states_val.to("cpu").numpy()

torch.cuda.empty_cache()

cls_test, hidden_states_test = get_embd_tr(test_tweets)
x_cls_test = cls_test.to("cpu")
x_hiddenstates_test = hidden_states_test.to("cpu").numpy()

#SVM


In [ ]:
import numpy as np

class HardMarginSVM:
    """
    Attributes
    ----------
    eta : float
    epoch : int
    random_state : int
    is_trained : bool
    num_samples : int
    num_features : int
    w : NDArray[float]
    b : float
    alpha : NDArray[float]

    Methods
    -------
    fit -> None
        Fitting parameter vectors for training data
    predict -> NDArray[int]
        Return predicted value
    """
    def __init__(self, eta=0.0000001, epoch=1000, random_state=5):
        self.eta = eta
        self.epoch = epoch
        self.random_state = random_state
        self.is_trained = False

    def fit(self, X, y):
        """
        Fitting parameter vectors for training data

        Parameters
        ----------
        X : NDArray[NDArray[float]]
        y : NDArray[float]
        """
        self.num_samples = X.shape[0]
        self.num_features = X.shape[1]
        self.w = np.zeros(self.num_features)
        self.b = 0
        rgen = np.random.RandomState(self.random_state)
        self.alpha = rgen.normal(loc=0.0, scale=0.01, size=self.num_samples)

        for _ in range(self.epoch):
            self._cycle(X, y)
        
        indexes_sv = [i for i in range(self.num_samples) if self.alpha[i] != 0]
        for i in indexes_sv:
            self.w += self.alpha[i] * y[i] * X[i]
        for i in indexes_sv:
            self.b += y[i] - (self.w @ X[i])
        self.b /= len(indexes_sv)
        self.is_trained = True

    def predict(self, X):
        """
        Return predicted value

        Parameters
        ----------
        X : NDArray[NDArray[float]]

        Returns
        -------
        result : NDArray[int]
        """
        if not self.is_trained:
            raise Exception('This model is not trained.')

        hyperplane = X @ self.w + self.b
        result = np.where(hyperplane > 0, 1, -1)
        return result
        
    def _cycle(self, X, y):
        """
        One cycle of gradient descent method

        Parameters
        ----------
        X : NDArray[NDArray[float]]
        y : NDArray[float]
        """
        y = y.reshape([-1, 1])
        H = (y @ y.T) * (X @ X.T)
        grad = np.ones(self.num_samples) - H @ self.alpha
        self.alpha += self.eta * grad
        self.alpha = np.where(self.alpha < 0, 0, self.alpha)

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_tr = sc.fit_transform(X_train_avg_w2v)
X_te = sc.fit_transform(X_test_avg_w2v)
y_tr = np.where(y_train==0, -1, 1)
y_te = np.where(y_test==0, -1, 1)

In [ ]:
hard_margin_svm = HardMarginSVM()


In [ ]:
hard_margin_svm.fit(X_tr, y_tr)


In [ ]:
predict = hard_margin_svm.predict(X_te)

In [ ]:
# Create confusion matrix
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, classification_report
print(confusion_matrix(y_te, predict))
# Display accuracy score
print(accuracy_score(y_te,predict))
# Display F1 score
print(classification_report(y_te,predict))
accuracy_score(y_te, predict)


SVM with library

In [ ]:
from sklearn import svm

sk_svm = svm.LinearSVC(C=0.0001, random_state=42)
sk_svm.fit(X_tr, y_tr)


In [ ]:
pre = sk_svm.predict(X_te)

In [ ]:
# Create confusion matrix
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, classification_report
print(confusion_matrix(y_te, pre))
# Display accuracy score
print(accuracy_score(y_te,pre))
# Display F1 score
print(classification_report(y_te,pre))
accuracy_score(y_te, pre)

# Random Forest

In [ ]:
# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
	correct = 0
	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct += 1
	return correct / float(len(actual)) * 100.0

In [ ]:
# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(train_set,test_set, algorithm, *args):
	predicted = algorithm(train_set, test_set, *args)
	# actual = [row[-1] for row in fold]
	# accuracy = accuracy_metric(actual, predicted)
	# scores.append(accuracy)
	return predicted

In [ ]:
# Split a dataset based on an attribute and an attribute value
def test_split(index, value, dataset):
	left, right = list(), list()
	for row in dataset:
		if row[index] < value:
			left.append(row)
		else:
			right.append(row)
	return left, right

In [ ]:
# Calculate the Gini index for a split dataset
def gini_index(groups, classes):
	# count all samples at split point
	n_instances = float(sum([len(group) for group in groups]))
	# sum weighted Gini index for each group
	gini = 0.0
	for group in groups:
		size = float(len(group))
		# avoid divide by zero
		if size == 0:
			continue
		score = 0.0
		# score the group based on the score for each class
		for class_val in classes:
			p = [row[-1] for row in group].count(class_val) / size
			score += p * p
		# weight the group score by its relative size
		gini += (1.0 - score) * (size / n_instances)
	return gini

In [ ]:
# Select the best split point for a dataset
def get_split(dataset, n_features):
	class_values = list(set(row[-1] for row in dataset))
	b_index, b_value, b_score, b_groups = 999, 999, 999, None
	features = list()
	while len(features) < n_features:
		index = randrange(len(dataset[0])-1)
		if index not in features:
			features.append(index)
	for index in features:
		for row in dataset:
			groups = test_split(index, row[index], dataset)
			gini = gini_index(groups, class_values)
			if gini < b_score:
				b_index, b_value, b_score, b_groups = index, row[index], gini, groups
	return {'index':b_index, 'value':b_value, 'groups':b_groups}

In [ ]:
# Create a terminal node value
def to_terminal(group):
	outcomes = [row[-1] for row in group]
	return max(set(outcomes), key=outcomes.count)

In [ ]:
# Create child splits for a node or make terminal
def split(node, max_depth, min_size, n_features, depth):
	left, right = node['groups']
	del(node['groups'])
	# check for a no split
	if not left or not right:
		node['left'] = node['right'] = to_terminal(left + right)
		return
	# check for max depth
	if depth >= max_depth:
		node['left'], node['right'] = to_terminal(left), to_terminal(right)
		return
	# process left child
	if len(left) <= min_size:
		node['left'] = to_terminal(left)
	else:
		node['left'] = get_split(left, n_features)
		split(node['left'], max_depth, min_size, n_features, depth+1)
	# process right child
	if len(right) <= min_size:
		node['right'] = to_terminal(right)
	else:
		node['right'] = get_split(right, n_features)
		split(node['right'], max_depth, min_size, n_features, depth+1)

In [ ]:
# Build a decision tree
def build_tree(train, max_depth, min_size, n_features):
	root = get_split(train, n_features)
	split(root, max_depth, min_size, n_features, 1)
	return root

In [ ]:
# Make a prediction with a decision tree
def predict(node, row):
	if row[node['index']] < node['value']:
		if isinstance(node['left'], dict):
			return predict(node['left'], row)
		else:
			return node['left']
	else:
		if isinstance(node['right'], dict):
			return predict(node['right'], row)
		else:
			return node['right']

In [ ]:
# Create a random subsample from the dataset with replacement
def subsample(dataset, ratio):
	sample = list()
	n_sample = round(len(dataset) * ratio)
	while len(sample) < n_sample:
		index = randrange(len(dataset))
		sample.append(dataset[index])
	return sample

In [ ]:
# Make a prediction with a list of bagged trees
def bagging_predict(trees, row):
	predictions = [predict(tree, row) for tree in trees]
	return max(set(predictions), key=predictions.count)

In [ ]:
# Random Forest Algorithm
def random_forest(train, test, max_depth, min_size, sample_size, n_trees, n_features):
	trees = list()
	for i in range(n_trees):
		sample = subsample(train, sample_size)
		tree = build_tree(sample, max_depth, min_size, n_features)
		trees.append(tree)
	predictions = [bagging_predict(trees, row) for row in test]
	return(predictions)


In [ ]:
my_dataset = X_train_avg_w2v.numpy().copy()

In [ ]:
data_train = []
for i in range(len(my_dataset)):
  item_list = list(my_dataset[i])
  print(item_list)
  item_list.append(y_train[i])
  print(item_list)
  data_train.append(item_list)
  # list(my_dataset[i]).append(y_train[i])
  


In [ ]:
my_dataset_test = X_test_avg_w2v.numpy().copy()

In [ ]:
data_test = []
for i in range(len(my_dataset_test)):
  item_list = list(my_dataset_test[i])
  print(item_list)
  item_list.append(y_test[i])
  print(item_list)
  data_test.append(item_list)
  # list(my_dataset[i]).append(y_train[i])

In [ ]:
# convert string attributes to integers

# evaluate algorithm
n_folds = 5
max_depth = 10
min_size = 1
sample_size = 1.0
# n_features = int(sqrt(len(dataset[0])-1))
n_features = 100
predicts = []
for n_trees in [1, 5, 10]:
	pre = evaluate_algorithm(data_train[:3000],data_test, random_forest, max_depth, min_size, sample_size, 1, n_features)
	print(pre)
	predicts.append(pre)
	# print('Trees: %d' % n_trees)
	# print('Scores: %s' % scores)
	# print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

In [ ]:
# Create confusion matrix
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, classification_report
for i in range(3):
  print("number", i)
  print(confusion_matrix(y_test[:500], predicts[i]))
  # Display accuracy score
  print(accuracy_score(y_test[:500], predicts[i]))
  # Display F1 score
  print(classification_report(y_test[:500], predicts[i]))
  accuracy_score(y_test[:500], predicts[i])
  print("\n")


Random Forest with library

In [ ]:
# Instantiate and fit a basic Random Forest model on top of the vectors
from sklearn.ensemble import RandomForestClassifier
random_f = RandomForestClassifier(criterion="gini", n_estimators=10, max_depth =10,min_samples_split = 0.1,max_features= 100,)
random_f_model = random_f.fit(X_train_avg_w2v, y_train)

In [ ]:
# Use the trained model to make predictions on the test data
y_pred = random_f_model.predict(X_test_avg_w2v)

In [ ]:
# Create confusion matrix
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, classification_report
print(confusion_matrix(y_test, y_pred))
# Display accuracy score
print(accuracy_score(y_test,y_pred))
# Display F1 score
print(classification_report(y_test,y_pred))
accuracy_score(y_test, y_pred)


## **Add Extra features**

# sentiment

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
 
# function to print sentiments
# of the sentence.
def sentiment_scores(sentence):
    sid_obj = SentimentIntensityAnalyzer()
    sentiment_dict = sid_obj.polarity_scores(sentence)

    # decide sentiment as positive(1), negative(-1) and neutral(0)
    if sentiment_dict['compound'] >= 0.05 :
        return 1
 
    elif sentiment_dict['compound'] <= - 0.05 :
        return -1
 
    else :
        return 0
 
 
 

In [ ]:
def Sentiment_data(tweets):
  sent_data = []
  for i  in range(len(tweets)):
    s = sentiment_scores(tweets[i])
    sent_data.append(s)
  return sent_data



In [ ]:
sent_data_train = Sentiment_data(train_tweets)
sent_data_val = Sentiment_data(val_tweets)
sent_data_test = Sentiment_data(test_tweets)



# Tweet length

In [ ]:
def Tweet_length(tweets):
  len_tweet = []
  for i in range(len(tweets)):
  l = len(tweets[i])
  len_tweet.append(l)

In [ ]:
len_train = Tweet_length(train_tweets)
len_val = Tweet_length(val_tweets)
len_test = Tweet_length(test_tweets)

# Neural networks

In [ ]:
# trasfromer embedding
sentence_length = x_hidden_states_train.shape[1]
feature_number =x_hidden_states_train.shape[2]


In [ ]:
# word2vec
sentence_length = X_train_w2v.shape[1]
feature_number = 8


In [ ]:
inputA = layers.Input(shape=(sentence_length,feature_number))


In [ ]:
model = Sequential()
model.add(inputA)

# LSTM
# model.add(LSTM(32, dropout=0.2,  return_sequences=False))

# GRU
# model.add(GRU(32, dropout=0.2,  return_sequences=False))

# Bi LSTM
# model.add(Bidirectional(LSTM(32, dropout=0.2,  return_sequences=False)))

# Bi GRU
model.add(Bidirectional(GRU(32, dropout=0.2,  return_sequences=False)))

model.add(layers.Dense(1, activation='sigmoid', name="dense"))

Model with sentiment and tweet length feature

In [ ]:
# define two sets of inputs
inputB = layers.Input(shape=(1,))
inputC = layers.Input(shape=(1,))

x = Bidirectional(GRU(32, dropout=0.2,  return_sequences=True))(inputA)
x = layers.Flatten()(x)
x = Dense(4, activation="relu")(x)
x = Model(inputs=inputA, outputs=x)

y = Dense(4,activation="relu")(inputB)
y = Model(inputs=inputB, outputs=y)

w = Dense(4, activation="relu")(inputC)
w = Model(inputs=inputC, outputs=w)

# combine the output of the three branches
combined = layers.concatenate([x.output, y.output, w.output])
# combined outputs
z = Dense(1, activation="sigmoid")(combined)

# our model will accept the inputs of the three branches and
# then output a single value
model = Model(inputs=[x.input, y.input, w.input], outputs=z)


In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=00.0001)
# opt=AdamW(learning_rate=00.0001, weight_decay = 0.000000001)
# from tensorflow_addons.optimizers import AdamW
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy',f1_m,precision_m, recall_m])


In [ ]:
# word2vec
history = model_nn.fit(x = x_hidden_states_train, y = y_train,epochs=50, validation_data=(x_hidden_states_val, y_val))

In [ ]:
print(confusion_matrix(y_test,pred))
print(classification_report(y_test,pred))
accuracy_score(y_test, pred)

In [ ]:
in2 = np.array(sent_data_train)
val_in2= np.array(sent_data_valid)


In [ ]:
in1=x_hidden_states_train
val_in1 =x_hidden_states_val


In [ ]:
in3=np.array(len_train)
val_in3 =np.array(len_val)


In [ ]:
test_in2 = np.array(sent_data_test)
test_in1 = x_hidden_states_test
# test_in3  =np.array(len_test)


In [ ]:
from tensorflow.keras.utils import Sequence
import numpy as np   

class DataGenerator(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return batch_x, batch_y

train_gen = DataGenerator([x_hiddenstates_train, in2, in3], y_train, 16)

val_gen = DataGenerator([x_hiddenstates_val, val_in2, val_in3],y_val, 16)


In [ ]:
# transformer
# x=[trainAttrX, trainImagesX], y=trainY,

# history = model.fit(train_gen,
#                     epochs=6,
#                     validation_data=test_gen)
history = model.fit(dataset_train,epochs=50, validation_data=(dataset_val))
# history = model.fit(x =[x_hiddenstates_train,in2,in3],y = y_train ,epochs=50, validation_data=([x_hiddenstates_val,val_in2,val_in3], y_val))

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')

# def plot_history(history):
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
x = range(1, len(acc) + 1)

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(x, acc, 'b', label='Training acc')
plt.plot(x, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.subplot(1, 2, 2)
plt.plot(x, loss, 'b', label='Training loss')
plt.plot(x, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()